In [ ]:
# 과대적합 피하기

In [ ]:
# 𝑙_1과 𝑙_2 규제 (계속)
# 𝑙_1 -> 불필요한 weight 0으로 만듦 -> 희소모델 -> 간단한 모델을 만들고 싶을 때 유용

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import sklearn
import matplotlib.pyplot as plt

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
pixel_means = X_train.mean(axis=0, keepdims=True) 
pixel_stds = X_train.std(axis=0, keepdims=True) 
X_train_scaled = (X_train - pixel_means) / pixel_stds 
X_valid_scaled = (X_valid - pixel_means) / pixel_stds 
X_test_scaled = (X_test - pixel_means) / pixel_stds

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
X_train.shape

(55000, 28, 28)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer=keras.regularizers.l2(0.01))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 18s 10ms/step - loss: 1.6334 - accuracy: 0.8124 - val_loss: 0.7240 - val_accuracy: 0.8346
Epoch 2/2
1719/1719 [==============================] - 19s 11ms/step - loss: 0.7207 - accuracy: 0.8252 - val_loss: 0.7059 - val_accuracy: 0.8328


In [ ]:
# functools 의 partial 사용 예

In [ ]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.1)) # 0.01 -> 1.0 -> 0.1

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 17s 10ms/step - loss: 3.8310 - accuracy: 0.7137 - val_loss: 1.5927 - val_accuracy: 0.7312
Epoch 2/2
1719/1719 [==============================] - 17s 10ms/step - loss: 1.5995 - accuracy: 0.7067 - val_loss: 1.5650 - val_accuracy: 0.7274


In [ ]:
# 드롭아웃
# 에포크 또는 스텝마다 서로 다른 고유한 모델이 훈련되는 효과를 발생시킴
# 최근 구글 논문 -> 극한의 상황에서도 작동을 하는지 테스트
# 훈련이 끝난 후에는 뉴런에 더는 드롭아웃을 적용하지 않는다.
# p = 0.9 -> 입력에 10 곱해서 사용 or 테스트 할때 입력값에 0.1곱하기


In [ ]:
# 입력에 10 곱하기(0.1로 나누기(1-p))
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))


Epoch 1/2
1719/1719 [==============================] - 22s 12ms/step - loss: 0.5755 - accuracy: 0.8020 - val_loss: 0.3675 - val_accuracy: 0.8682
Epoch 2/2
1719/1719 [==============================] - 18s 10ms/step - loss: 0.4236 - accuracy: 0.8460 - val_loss: 0.3376 - val_accuracy: 0.8768


In [ ]:
# 알파 드롭 아웃 실습

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.AlphaDropout(rate=0.4), # 0.2 -> 0.4
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.4),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.4),
    keras.layers.Dense(10, activation="softmax")
])
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
n_epochs = 5
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))


Epoch 1/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.8939 - accuracy: 0.6711 - val_loss: 1.1897 - val_accuracy: 0.7882
Epoch 2/5
1719/1719 [==============================] - 9s 5ms/step - loss: 0.7304 - accuracy: 0.7292 - val_loss: 0.9898 - val_accuracy: 0.8196
Epoch 3/5
1719/1719 [==============================] - 9s 5ms/step - loss: 0.6951 - accuracy: 0.7429 - val_loss: 1.0834 - val_accuracy: 0.8024
Epoch 4/5
1719/1719 [==============================] - 9s 5ms/step - loss: 0.6689 - accuracy: 0.7525 - val_loss: 0.9211 - val_accuracy: 0.8240
Epoch 5/5
1719/1719 [==============================] - 9s 5ms/step - loss: 0.6562 - accuracy: 0.7575 - val_loss: 0.8840 - val_accuracy: 0.8158
